In [105]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-ml-w5p1/trainX.csv
/kaggle/input/2024-ml-w5p1/testX.csv
/kaggle/input/2024-ml-w5p1/trainY.csv
/kaggle/input/2024-ml-w5p1/submit.csv


In [106]:
#랜덤시드 고정
import random
import os

seed = 42

random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [107]:
#데이터셋 로드
trainX = pd.read_csv('/kaggle/input/2024-ml-w5p1/trainX.csv')
trainY = pd.read_csv('/kaggle/input/2024-ml-w5p1/trainY.csv')
testX = pd.read_csv('/kaggle/input/2024-ml-w5p1/testX.csv')
submit = pd.read_csv('/kaggle/input/2024-ml-w5p1/submit.csv')

In [108]:
#64차원의 1-Dim 벡터로 가공된 데이터셋 확인
print(f'shape of train data: {trainX.shape}')
trainX.head()

shape of train data: (6000, 64)


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,13.934082,-3.077109,-13.515020,-0.844135,3.097764,0.154635,5.615488,-8.032149,2.776431,1.469208,...,-0.834850,0.474123,-1.481585,-2.168011,-0.472213,1.542271,0.356707,0.530720,0.171594,-0.322960
1,18.757893,-0.304454,1.757282,10.702372,5.530047,-9.571358,9.296905,-2.858612,0.027188,3.768508,...,0.941480,1.776686,0.905478,-1.641744,0.734237,1.231368,0.179600,1.700099,-0.001509,1.077432
2,-8.063476,-3.259291,-16.577340,-5.497738,-6.616893,4.916349,-10.537728,5.398572,-1.091936,-2.561823,...,1.976522,-0.962800,-0.409117,0.497765,1.440391,-0.513098,-0.477530,2.724299,-0.663966,-1.755266
3,3.128894,16.911400,-10.434631,1.177685,3.228336,-1.875124,8.374058,-0.885263,5.068379,-6.400661,...,1.624901,0.775439,3.694294,0.438467,-0.148669,0.227370,3.004657,0.440074,-0.087064,1.925870
4,-15.744248,-1.022630,0.004898,6.656867,-2.534440,-8.309071,-1.379099,4.352854,8.783724,2.650707,...,0.231652,-1.151704,0.354604,0.752026,-1.192524,-0.910384,1.549595,-2.284629,0.672590,-0.606422


In [109]:
#trainY 데이터셋 확인
display(trainY.head())

print(f'columns: {trainY.columns}')
print(f'labels : {trainY.Category.unique()}')

,Id,Category
0,0,edge
1,1,edge
2,2,edge
3,3,edge
4,4,edge


columns: Index(['Id', 'Category'], dtype='object')
labels : ['edge' 'smooth' 'spiral']


In [110]:
#id drop
trainY.drop(['Id'], axis=1, inplace=True)

In [111]:
#데이터 타입 확인
print(type(trainY['Category']))
print(type(trainY['Category'].values))

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [112]:
#labelEncoder를 이용해 정수로 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(trainY['Category'].values))
y = y.values.ravel() # labels.squeeze()

In [113]:
#학습용 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(trainX, y, test_size=0.2, random_state=seed, stratify=y)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(4800, 64)
(1200, 64)
(4800,)
(1200,)


In [114]:
#표준화
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)
test_std = sc.transform(testX)

In [115]:
#Logistic regression
from sklearn.linear_model import LogisticRegression

Logit = LogisticRegression(random_state=seed, max_iter=1000)
#Logit.fit(x_train_std, y_train)
Logit.fit(x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [116]:
#예측
#y_train_pred = Logit.predict(x_train_std)
#y_test_pred = Logit.predict(x_test_std)
y_train_pred = Logit.predict(x_train)
y_test_pred = Logit.predict(x_test)

In [117]:
#accuracy score 측정
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test, y_test_pred))

0.8052083333333333
0.7825


In [118]:
#Confusion matrix (정확도 행렬)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_test_pred))

[[331  40  29]
 [ 25 319  56]
 [ 26  85 289]]


In [123]:
#test 예측
Logit.fit(trainX, y)
test_pred = Logit.predict(testX)
test_pred = le.inverse_transform(test_pred)
test_pred

array(['edge', 'smooth', 'spiral', ..., 'edge', 'spiral', 'smooth'],
      dtype=object)

In [124]:
# 제출 포맷 확인
submit.head()

,Id,Category
0,0,edge
1,1,smooth
2,2,spiral
3,3,spiral
4,4,edge


In [121]:
# submit 저장
submit['Category'] = test_pred
submit.to_csv('Baseline.csv', index = False)

In [122]:
submit.head()

,Id,Category
0,0,edge
1,1,smooth
2,2,spiral
3,3,spiral
4,4,edge
